# Ridge

In [1]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme(
    context="paper", 
    style="whitegrid", 
    font_scale=1.2,
    rc={'figure.figsize': (10, 10), 'figure.dpi': 300}
)

## Get Data

In [2]:
from common.data import get_data

X, Y, demographics, population = get_data(wisc_level=5)
ages = demographics['Age']

print(f'X: {X.shape} | Y: {len(Y.keys())} | Age: {ages.shape} | Population: {population}')

X: (380, 34716) | Y: 6 | Age: (380,) | Population: adhd


In [6]:
from common.scoring import unimetric_scorer, multimetric_scorer, N_PERM, SCORING, RKF_10_10
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import cross_validate, permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

## Run for one target, one age bin

In [4]:
selected_target = "WISC_FSIQ"
y = Y[selected_target]

print(f'y: {y.shape}')

y: (380,)


In [91]:
from common.binning import bin_by_age

bins = bin_by_age(X, y, ages, y)
bin_1, bin_2, bin_3 = bins[0], bins[1], bins[2]
print(f'Bin 1: {bin_1[0].shape} | Bin 2: {bin_2[0].shape} | Bin 3: {bin_3[0].shape}')

Bin 1: (117, 34716) | Bin 2: (156, 34716) | Bin 3: (117, 34716)


### Set up the model pipeline, metrics, and cross-validation approach

In [7]:
%%time
# X_cv = bin_1[0]
# y_cv = bin_1[1]
X_cv = X
y_cv = y
age_group = 'all'

estimators = [StandardScaler(), RidgeCV(alphas=[a for a in range(5000, 55000, 5000)], 
                                        scoring=unimetric_scorer, cv=RKF_10_10)]
pipe = make_pipeline(*estimators)
pipe.fit(X_cv, y_cv)
ridge_cv = pipe['ridgecv']

print(f'Target: {selected_target} | Alpha: {ridge_cv.alpha_} | Score: {ridge_cv.best_score_:.2f}')

estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
pipe = make_pipeline(*estimators)

Target: WISC_FSIQ | Alpha: 15000 | Score: 0.39
CPU times: user 8min 36s, sys: 14.8 s, total: 8min 51s
Wall time: 53.4 s


In [8]:
%%time
# X_cv = bin_1[0]
# y_cv = bin_1[1]
X_cv = X
y_cv = y
age_group = 'all'

estimators = [StandardScaler(), MultiOutputRegressor(
    RidgeCV(alphas=[a for a in range(5000, 55000, 5000)], scoring=regression_scorer, cv=rkf))]
pipe = make_pipeline(*estimators)
pipe.fit(X_cv, y_cv)
ridge_cvs = pipe['multioutputregressor'].estimators_

for target, ridge_cv in zip(WISC_LEVEL[5], ridge_cvs):
    print(f'Target: {target} | Alpha: {ridge_cv.alpha_} | Score: {ridge_cv.best_score_:.2f}')

ValueError: y must have at least two dimensions for multi-output regression but has only one.

### Run cross-validation

In [11]:
%%time
estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
pipe = make_pipeline(*estimators)

scores = cross_validate(pipe, X_cv, y_cv, cv=RKF_10_10, scoring=multimetric_scorer, n_jobs=-1, 
                        return_train_score=True, return_estimator=True)

coefs = np.array([estimator['ridge'].coef_ for estimator in scores['estimator']])
avg_coef = np.mean(coefs, axis=0)
intercepts = np.array([estimator['ridge'].intercept_ for estimator in scores['estimator']])
avg_inte = np.mean(intercepts, axis=0)

print(f'ridge_{population}_{selected_target}_{age_group}')
for metric in scoring:
    metric_values = scores[metric]
    print(f'Avg {metric}: {np.mean(metric_values):.3f}')

ridge_adhd_WISC_FSIQ_all
Avg train_score: 0.890
Avg test_score: 0.348
CPU times: user 901 ms, sys: 283 ms, total: 1.18 s
Wall time: 6.76 s


In [38]:
from os.path import join
from common.paths import RIDGE_WEIGHTS

coef_fn = f'ridge_{population}_{selected_target}_{age_group}_coef.npy'
inte_fn = f'ridge_{population}_{selected_target}_{age_group}_inte.npy'

np.save(join(RIDGE_WEIGHTS, coef_fn), avg_coef)
np.save(join(RIDGE_WEIGHTS, inte_fn), avg_inte)

### Transfer model weights

In [ ]:
from os.path import join
from common.paths import RIDGE_WEIGHTS

weight_pop = 'adhd'
weight_tar = 'WISC_FSIQ'
weight_age = 'all'
coef_fn = f'ridge_{weight_pop}_{weight_tar}_{weight_age}_coef.npy'
inte_fn = f'ridge_{weight_pop}_{weight_tar}_{weight_age}_inte.npy'

alpha = 5000
intercept = 108.534744

ridge = Ridge(alpha=alpha)
ridge.coef_ = np.load(join(RIDGE_WEIGHTS, coef_fn))
ridge.intercept_ = np.load(join(RIDGE_WEIGHTS, inte_fn))

print('Score:', regression_scorer(ridge, X, y))

#### ICC

In [34]:
%%time
import pingouin as pg

icc_data = pd.DataFrame(coefs).melt(var_name='connection', value_name='weight', ignore_index=False)
icc_data['cv_run_num'] = icc_data.index
icc = pg.intraclass_corr(data=icc_data, targets='connection', raters='cv_run_num', ratings='weight').round(3)
icc.set_index("Type")

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.897,873.76,34715,3436884,0.0,"[0.9, 0.9]"
ICC2,Single random raters,0.897,873.88,34715,3436785,0.0,"[0.9, 0.9]"
ICC3,Single fixed raters,0.897,873.88,34715,3436785,0.0,"[0.9, 0.9]"
ICC1k,Average raters absolute,0.999,873.76,34715,3436884,0.0,"[1.0, 1.0]"
ICC2k,Average random raters,0.999,873.88,34715,3436785,0.0,"[1.0, 1.0]"
ICC3k,Average fixed raters,0.999,873.88,34715,3436785,0.0,"[1.0, 1.0]"


### Run permutation statistic

In [37]:
%%time
score, _, pvalue = permutation_test_score(
    pipe, X_cv, y_cv, cv=rkf, scoring=regression_scorer, n_permutations=n_perm, n_jobs=-1)

print(f'ridge_{population}_{selected_target}_{age_group}')
print(f'Alpha: {ridge_cv.alpha_} | Score: {score:.2f} | p-value: {pvalue:.4f}')

ridge_healthy_WISC_PSI_all
Alpha: 5000 | Score: -0.04 | p-value: 0.6268
CPU times: user 1min 13s, sys: 2.12 s, total: 1min 15s
Wall time: 1h 5min 49s


## Run for all targets, all age bins

In [9]:
%%time
from common.binning import bin_by_age
from common.wisc import FSIQ, PRIMARY_INDICES
from common.paths import RIDGE_RESULTS

results = []
targets = FSIQ + PRIMARY_INDICES

for target in targets:
    y = Y[target]
    bins = bin_by_age(X, y, ages, y)
    bin_1, bin_2, bin_3 = bins[0], bins[1], bins[2]
    X_all = [X, bin_1[0], bin_2[0], bin_3[0]]
    y_all = [y, bin_1[1], bin_2[1], bin_3[1]]
    bin_labels = ["All  ", "Bin 1", "Bin 2", "Bin 3"]
#     X_all = [X]
#     y_all = [y]
#     bin_labels = ["All  "]
    
    for X_cv, y_cv, bin_label in zip(X_all, y_all, bin_labels):
        # Find best alpha
        estimators = [StandardScaler(), RidgeCV(alphas=[a for a in range(1000, 21000, 3000)], 
                                        scoring=regression_scorer, cv=rkf)]
        pipe = make_pipeline(*estimators).fit(X_cv, y_cv)
        ridge_cv = pipe['ridgecv']
        
        # Do permutation test
        estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
        pipe = make_pipeline(*estimators)
        
        score, _, pvalue = permutation_test_score(
            pipe, X_cv, y_cv, cv=rkf, scoring=regression_scorer, n_permutations=n_perm, n_jobs=-1)
        results.append({    
            'Model': 'ridge',
            'Population': population,
            'Target': target,
            'Bin': bin_label,
            'Alpha': ridge_cv.alpha_,
            'Score': score,
            'P-value': pvalue,
        })
        print(results[-1])
        
results_df = pd.DataFrame(results)
display(results_df)
results_df.to_csv(RIDGE_RESULTS)

{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'All  ', 'Alpha': 19000, 'Score': 0.37543143020424613, 'P-value': 0.0003332222592469177}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'Bin 1', 'Alpha': 10000, 'Score': 0.35711821000437227, 'P-value': 0.001999333555481506}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'Bin 2', 'Alpha': 1000, 'Score': 0.3902874552871233, 'P-value': 0.0003332222592469177}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'Bin 3', 'Alpha': 19000, 'Score': -0.030799570667886605, 'P-value': 0.5898033988670444}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_VSI', 'Bin': 'All  ', 'Alpha': 19000, 'Score': 0.32780173019528824, 'P-value': 0.0003332222592469177}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_VSI', 'Bin': 'Bin 1', 'Alpha': 19000, 'Score': 0.29324738447125887, 'P-value': 0.006997667444185272}
{'Model': 'ridge', 'Population': 'adhd', 'Targe

,Model,Population,Target,Bin,Alpha,Score,P-value
0,ridge,adhd,WISC_FSIQ,All,19000,0.375431,0.000333
1,ridge,adhd,WISC_FSIQ,Bin 1,10000,0.357118,0.001999
2,ridge,adhd,WISC_FSIQ,Bin 2,1000,0.390287,0.000333
3,ridge,adhd,WISC_FSIQ,Bin 3,19000,-0.030800,0.589803
4,ridge,adhd,WISC_VSI,All,19000,0.327802,0.000333
5,ridge,adhd,WISC_VSI,Bin 1,19000,0.293247,0.006998
6,ridge,adhd,WISC_VSI,Bin 2,19000,0.320526,0.001000
7,ridge,adhd,WISC_VSI,Bin 3,19000,-0.010053,0.540153
8,ridge,adhd,WISC_VCI,All,10000,0.388630,0.000333
9,ridge,adhd,WISC_VCI,Bin 1,1000,0.280998,0.010330


CPU times: user 1h 43min 29s, sys: 2min 52s, total: 1h 46min 21s
Wall time: 1d 20h 41min 56s
